In [1]:
import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [3]:
movies = movies.merge(credits,on="title")

In [4]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [5]:
movies.dropna(inplace=True)  # remove all null values data

In [6]:
movies.duplicated().sum()   # check duplicates

0

In [7]:
def HelperForProcessing(obj):
    listData = []
    for data in ast.literal_eval(obj):
        listData.append(data['name'])
    return listData

In [8]:
import ast

In [9]:
movies['genres'] = movies['genres'].apply(HelperForProcessing)

In [10]:
movies['keywords'] = movies['keywords'].apply(HelperForProcessing)

In [11]:
def get5Cast(obj):
    listData = []
    temp = 0
    for data in ast.literal_eval(obj):
        if temp > 5:
            break
        listData.append(data['name'])
        temp+=1
    return listData

In [12]:
movies['cast'] = movies['cast'].apply(get5Cast)

In [13]:
def getDirector(obj):
    listdata = []
    for data in ast.literal_eval(obj):
        if data['job'] == 'Director':
            listdata.append(data['name'])
            break
    return listdata

In [14]:
movies['crew'] = movies['crew'].apply(getDirector)

In [15]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [16]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [17]:
movies['tags'] = movies['overview'] + movies['genres']+ movies['keywords'] + movies['cast'] + movies['crew']

In [18]:
data_frame = movies[['movie_id','title','tags']]

In [19]:
data_frame['tags'] = data_frame['tags'].apply(lambda x:" ".join(x))

C:\Users\praka\AppData\Local\Temp\ipykernel_9084\674701807.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame['tags'] = data_frame['tags'].apply(lambda x:" ".join(x))


In [20]:
import nltk

In [21]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [22]:
def stem(text):
    data = []
    for word in text.split():
        data.append(ps.stem(word))
    
    return " ".join(data)

In [23]:
data_frame['tags'] = data_frame['tags'].apply(stem)

C:\Users\praka\AppData\Local\Temp\ipykernel_9084\2955560100.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame['tags'] = data_frame['tags'].apply(stem)


In [24]:
data_frame['tags'] = data_frame['tags'].apply(lambda x: x.lower())

C:\Users\praka\AppData\Local\Temp\ipykernel_9084\3079620373.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame['tags'] = data_frame['tags'].apply(lambda x: x.lower())


In [25]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [26]:
vectors = cv.fit_transform(data_frame['tags']).toarray()

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
similarity = cosine_similarity(vectors)

In [29]:
sorted(list(enumerate(similarity[1])),reverse=True,key=lambda x: x[1])[1:6]

[(12, 0.420210157631365),
 (199, 0.2684624220856097),
 (17, 0.23923566684866998),
 (3572, 0.20134681656420733),
 (216, 0.19344776406574832)]

In [30]:
def recommended(movie):
    movie_index = data_frame[data_frame['title'] == movie].index[0]
    distance = similarity[movie_index]
    movies_list = sorted(list(enumerate(distance)),reverse=True,key=lambda x: x[1])[1:6]
    
    for i in movies_list:
        print(data_frame.iloc[i[0]].title)
    

In [32]:
recommended('Batman Begins')

The Dark Knight
Batman
Batman
The Dark Knight Rises
Rockaway


In [168]:
import pickle

In [169]:
pickle.dump(data_frame,open('test.pkl','wb'))

In [170]:
pickle.dump(similarity,open('similarity.pkl','wb'))